In [ ]:
# --- Importação das bibliotecas ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

In [ ]:
# Seleção de Atributos para Modelo de Classificação

## Introdução
Este trabalho tem como objetivo a utilização de algoritmos de seleção de atributos como parte de um processo de Mineração de Dados para tarefas de classificação. O dataset Airline Passenger Satisfaction contém informações sobre passageiros e queremos prever a satisfação dos clientes (satisfied ou neutral/dissatisfied).

### Base de dados
O conjunto de dados Airline Passenger Satisfaction contém avaliações de diferentes aspectos da experiência de voo por passageiros. Os atributos incluem:
- Informações demográficas: Gênero, idade, tipo de cliente
- Detalhes do voo: Classe, tipo de viagem
- Avaliações de serviço: Limpeza, conforto do assento, serviço de bordo, etc.
- Atrasos em chegadas e partidas

### Objetivo
Desenvolver um modelo de classificação preciso e interpretável para prever a satisfação dos passageiros, utilizando técnicas de seleção de atributos para melhorar a performance e identificar os fatores mais importantes que influenciam a satisfação.

In [ ]:
# --- Carregamento dos dados ---
dados = pd.read_csv('/content/drive/MyDrive/airline_Passenger_Satisfaction.csv')
pd.set_option('display.max_columns', None)

# Exclusão dos atributos irrelevantes
dados.drop(columns=['Unnamed: 0', 'id'], axis=1, inplace=True)

# Exibir as primeiras linhas do dataset
print("Primeiras 5 linhas do dataset:")
print(dados.head())

# Informações sobre o dataset
print("\nInformações do dataset:")
print(dados.info())

# Estatísticas descritivas
print("\nEstatísticas descritivas:")
print(dados.describe())

In [ ]:
## Pré-processamento e exploração dos dados
Nesta seção, vamos explorar os dados para entender melhor suas características e relações, além de fazer o tratamento necessário para prepará-los para a modelagem.

In [ ]:
# --- Análise exploratória ---

# Verificar valores ausentes
ausentes = pd.DataFrame({
    'Quantidade': dados.isnull().sum(),
    'Freq. Relativa': dados.isnull().sum() / len(dados) * 100
})
print("Valores ausentes:")
print(ausentes[ausentes['Quantidade'] > 0])

# Tratamento dos valores ausentes: substituição dos valores nulos no atributo 'Arrival Delay in Minutes' pela média
# Atribuição da média por classe
media_satisfied = dados.loc[dados['satisfaction'] == 'satisfied', 'Arrival Delay in Minutes'].mean()
media_neutral_dissatisfied = dados.loc[dados['satisfaction'] == 'neutral or dissatisfied', 'Arrival Delay in Minutes'].mean()

# Substituição dos valores nulos pela média de acordo com a classe
dados.loc[(dados['satisfaction'] == 'satisfied') & (dados['Arrival Delay in Minutes'].isnull()), 'Arrival Delay in Minutes'] = media_satisfied
dados.loc[(dados['satisfaction'] == 'neutral or dissatisfied') & (dados['Arrival Delay in Minutes'].isnull()), 'Arrival Delay in Minutes'] = media_neutral_dissatisfied

# Verificar distribuição da variável alvo
plt.figure(figsize=(8, 6))
sns.countplot(x='satisfaction', data=dados)
plt.title('Distribuição da Satisfação dos Passageiros')
plt.show()

# Transformação das variáveis categóricas
lista_categoricas = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']
for coluna in lista_categoricas:
    if coluna != 'satisfaction':  # Não codificar a variável alvo ainda
        le = LabelEncoder()
        dados[coluna] = le.fit_transform(dados[coluna])
        print(f"{coluna} - mapeamento: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# Visualizar a matriz de correlação
plt.figure(figsize=(14, 12))
dados_numericos = dados.drop(columns=['satisfaction'])
correlacao = dados_numericos.corr()
sns.heatmap(correlacao, annot=True, cmap="Blues", fmt=".2f")
plt.title('Matriz de Correlação')
plt.tight_layout()
plt.show()

# Boxplots para visualizar a distribuição dos atributos numéricos por classe
atributos_numericos = dados.select_dtypes(include=np.number).columns[:5]  # Primeiros 5 atributos numéricos
plt.figure(figsize=(15, 10))
for i, feature in enumerate(atributos_numericos):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='satisfaction', y=feature, data=dados)
    plt.title(f'{feature} por Satisfação')
plt.tight_layout()
plt.show()

In [ ]:
# --- Análise exploratória ---

# Verificar valores ausentes
ausentes = pd.DataFrame({
    'Quantidade': dados.isnull().sum(),
    'Freq. Relativa': dados.isnull().sum() / len(dados) * 100
})
print("Valores ausentes:")
print(ausentes[ausentes['Quantidade'] > 0])

# Tratamento dos valores ausentes: substituição dos valores nulos no atributo 'Arrival Delay in Minutes' pela média
# Atribuição da média por classe
media_satisfied = dados.loc[dados['satisfaction'] == 'satisfied', 'Arrival Delay in Minutes'].mean()
media_neutral_dissatisfied = dados.loc[dados['satisfaction'] == 'neutral or dissatisfied', 'Arrival Delay in Minutes'].mean()

# Substituição dos valores nulos pela média de acordo com a classe
dados.loc[(dados['satisfaction'] == 'satisfied') & (dados['Arrival Delay in Minutes'].isnull()), 'Arrival Delay in Minutes'] = media_satisfied
dados.loc[(dados['satisfaction'] == 'neutral or dissatisfied') & (dados['Arrival Delay in Minutes'].isnull()), 'Arrival Delay in Minutes'] = media_neutral_dissatisfied

# Verificar distribuição da variável alvo
plt.figure(figsize=(8, 6))
sns.countplot(x='satisfaction', data=dados)
plt.title('Distribuição da Satisfação dos Passageiros')
plt.show()

# Transformação das variáveis categóricas
lista_categoricas = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']
for coluna in lista_categoricas:
    if coluna != 'satisfaction':  # Não codificar a variável alvo ainda
        le = LabelEncoder()
        dados[coluna] = le.fit_transform(dados[coluna])
        print(f"{coluna} - mapeamento: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# Visualizar a matriz de correlação
plt.figure(figsize=(14, 12))
dados_numericos = dados.drop(columns=['satisfaction'])
correlacao = dados_numericos.corr()
sns.heatmap(correlacao, annot=True, cmap="Blues", fmt=".2f")
plt.title('Matriz de Correlação')
plt.tight_layout()
plt.show()

# Boxplots para visualizar a distribuição dos atributos numéricos por classe
atributos_numericos = dados.select_dtypes(include=np.number).columns[:5]  # Primeiros 5 atributos numéricos
plt.figure(figsize=(15, 10))
for i, feature in enumerate(atributos_numericos):
    plt.subplot(2, 3, i+1)
    sns.boxplot(x='satisfaction', y=feature, data=dados)
    plt.title(f'{feature} por Satisfação')
plt.tight_layout()
plt.show()

In [ ]:
## Divisão em conjunto de treinamento e teste
Dividimos os dados em conjuntos de treinamento (80%) e teste (20%) para avaliar o desempenho do modelo em dados não vistos.

In [ ]:
# --- Divisão dos dados ---
# Codificar a variável alvo para o treinamento
le_target = LabelEncoder()
Y = le_target.fit_transform(dados['satisfaction'])
X = dados.drop('satisfaction', axis=1)

# Dividir em treino e teste (80% treino, 20% teste)
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(
    X, Y, test_size=validation_size, random_state=seed, stratify=Y)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_validation.shape[0]} amostras")

In [ ]:
# --- Divisão dos dados ---
# Codificar a variável alvo para o treinamento
le_target = LabelEncoder()
Y = le_target.fit_transform(dados['satisfaction'])
X = dados.drop('satisfaction', axis=1)

# Dividir em treino e teste (80% treino, 20% teste)
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(
    X, Y, test_size=validation_size, random_state=seed, stratify=Y)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_validation.shape[0]} amostras")

In [ ]:
# --- Criação e treinamento dos modelos iniciais ---
# Definir métricas de avaliação
scoring = 'accuracy'

# Algoritmos
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr', max_iter=1000)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', LinearSVC(max_iter=10000)))

# Avaliação de cada modelo com validação cruzada
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = '%s: %f (%f)' % (name, cv_results.mean(), cv_results.std())
    print(msg)

# Comparação dos Algoritmos
plt.figure(figsize=(10, 6))
plt.boxplot(results)
plt.title('Comparação dos Algoritmos - Modelos Iniciais')
plt.xticks(np.arange(1, len(names) + 1), names)
plt.ylabel('Accuracy')
plt.show()

# Avaliar os modelos no conjunto de validação
modelos_dict = {}
resultados_iniciais = pd.DataFrame(columns=['Modelo', 'Acurácia', 'Precisão', 'Recall', 'F1-Score'])

for i, (name, model) in enumerate(models):
    # Treinar o modelo
    model.fit(X_train, Y_train)
    modelos_dict[name] = model
    
    # Fazer previsões
    predictions = model.predict(X_validation)
    
    # Calcular métricas
    acc = accuracy_score(Y_validation, predictions)
    report = classification_report(Y_validation, predictions, output_dict=True)
    
    # Adicionar resultados
    resultados_iniciais.loc[i] = [name, acc, 
                                 report['weighted avg']['precision'], 
                                 report['weighted avg']['recall'], 
                                 report['weighted avg']['f1-score']]

# Mostrar resultados
print("Resultados dos modelos iniciais:")
print(resultados_iniciais)

# Selecionar o melhor modelo inicial para comparação posterior
melhor_modelo_idx = resultados_iniciais['Acurácia'].idxmax()
melhor_modelo_inicial = resultados_iniciais.loc[melhor_modelo_idx, 'Modelo']
print(f"Melhor modelo inicial: {melhor_modelo_inicial} com acurácia de {resultados_iniciais.loc[melhor_modelo_idx, 'Acurácia']:.4f}")

In [ ]:
## Seleção de Atributos
Agora vamos aplicar diferentes técnicas de seleção de atributos para identificar as características mais relevantes para o modelo.

In [ ]:
# --- 1. SelectKBest (baseado em qui-quadrado) ---
k = 10  # número de atributos a serem selecionados
selector_kbest = SelectKBest(score_func=chi2, k=k)
X_train_kbest = selector_kbest.fit_transform(X_train, Y_train)
X_validation_kbest = selector_kbest.transform(X_validation)

# Obter os atributos selecionados
atributos_kbest = X.columns[selector_kbest.get_support()]
print("Atributos selecionados pelo SelectKBest:")
print(list(atributos_kbest))

# --- 2. Recursive Feature Elimination (RFE) ---
estimator = RandomForestClassifier(random_state=seed)
selector_rfe = RFE(estimator, n_features_to_select=k)
X_train_rfe = selector_rfe.fit_transform(X_train, Y_train)
X_validation_rfe = selector_rfe.transform(X_validation)

# Obter os atributos selecionados pelo RFE
atributos_rfe = X.columns[selector_rfe.get_support()]
print("\nAtributos selecionados pelo RFE:")
print(list(atributos_rfe))

# --- 3. Importância de atributos via RandomForest ---
rf = RandomForestClassifier(random_state=seed)
rf.fit(X_train, Y_train)

# Obter importâncias
importancias = pd.Series(rf.feature_importances_, index=X.columns)
atributos_rf = importancias.sort_values(ascending=False)[:k].index.tolist()

print("\nAtributos selecionados pelo Random Forest:")
print(atributos_rf)

# Extrair subset do conjunto de dados com os atributos selecionados por RF
X_train_rf = X_train[atributos_rf]
X_validation_rf = X_validation[atributos_rf]

# Visualizar importância dos atributos
plt.figure(figsize=(12, 10))

plt.subplot(3, 1, 1)
plt.bar(atributos_kbest, selector_kbest.scores_[selector_kbest.get_support()])
plt.title('Importância dos Atributos - SelectKBest')
plt.xticks(rotation=90)

plt.subplot(3, 1, 2)
plt.bar(atributos_rfe, range(len(atributos_rfe)))
plt.title('Atributos Selecionados - RFE')
plt.xticks(rotation=90)

plt.subplot(3, 1, 3)
plt.bar(X.columns, rf.feature_importances_)
plt.title('Importância de Atributos - Random Forest')
plt.xticks(rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
## Treinamento e avaliação do modelo com seleção de atributos
Agora vamos treinar modelos utilizando apenas os atributos selecionados e comparar seu desempenho.

In [ ]:
# --- Treinar modelos com atributos selecionados ---
resultados_selecao = pd.DataFrame(columns=['Método', 'Modelo', 'Acurácia', 'Precisão', 'Recall', 'F1-Score'])
row_idx = 0

# Usar o melhor modelo inicial para comparar as técnicas de seleção de atributos
melhor_modelo = [model for name, model in models if name == melhor_modelo_inicial][0]

# Modelo com SelectKBest
modelo_kbest = type(melhor_modelo)()
if isinstance(modelo_kbest, LinearSVC):
    modelo_kbest = type(melhor_modelo)(max_iter=10000)
modelo_kbest.fit(X_train_kbest, Y_train)
pred_kbest = modelo_kbest.predict(X_validation_kbest)
acc_kbest = accuracy_score(Y_validation, pred_kbest)
report_kbest = classification_report(Y_validation, pred_kbest, output_dict=True)
resultados_selecao.loc[row_idx] = ['SelectKBest', melhor_modelo_inicial, 
                                  acc_kbest,
                                  report_kbest['weighted avg']['precision'],
                                  report_kbest['weighted avg']['recall'],
                                  report_kbest['weighted avg']['f1-score']]
row_idx += 1

# Modelo com RFE
modelo_rfe = type(melhor_modelo)()
if isinstance(modelo_rfe, LinearSVC):
    modelo_rfe = type(melhor_modelo)(max_iter=10000)
modelo_rfe.fit(X_train_rfe, Y_train)
pred_rfe = modelo_rfe.predict(X_validation_rfe)
acc_rfe = accuracy_score(Y_validation, pred_rfe)
report_rfe = classification_report(Y_validation, pred_rfe, output_dict=True)
resultados_selecao.loc[row_idx] = ['RFE', melhor_modelo_inicial,
                                  acc_rfe, 
                                  report_rfe['weighted avg']['precision'],
                                  report_rfe['weighted avg']['recall'],
                                  report_rfe['weighted avg']['f1-score']]
row_idx += 1

# Modelo com Random Forest Importance
modelo_rf = type(melhor_modelo)()
if isinstance(modelo_rf, LinearSVC):
    modelo_rf = type(melhor_modelo)(max_iter=10000)
modelo_rf.fit(X_train_rf, Y_train)
pred_rf = modelo_rf.predict(X_validation_rf)
acc_rf = accuracy_score(Y_validation, pred_rf)
report_rf = classification_report(Y_validation, pred_rf, output_dict=True)
resultados_selecao.loc[row_idx] = ['RandomForest', melhor_modelo_inicial,
                                  acc_rf, 
                                  report_rf['weighted avg']['precision'],
                                  report_rf['weighted avg']['recall'],
                                  report_rf['weighted avg']['f1-score']]
row_idx += 1

# Adicionar resultados do modelo original para comparação
original_acc = resultados_iniciais.loc[melhor_modelo_idx, 'Acurácia']
original_precision = resultados_iniciais.loc[melhor_modelo_idx, 'Precisão']
original_recall = resultados_iniciais.loc[melhor_modelo_idx, 'Recall']
original_f1 = resultados_iniciais.loc[melhor_modelo_idx, 'F1-Score']
resultados_selecao.loc[row_idx] = ['Original (Todos)', melhor_modelo_inicial,
                                  original_acc, original_precision, original_recall, original_f1]

# Mostrar resultados
print("Resultados com seleção de atributos:")
print(resultados_selecao)

# Visualizar comparação dos resultados
plt.figure(figsize=(12, 8))
sns.barplot(x='Método', y='Acurácia', data=resultados_selecao)
plt.title('Comparação de Acurácia entre Métodos de Seleção de Atributos')
plt.ylim(0.8, 1.0)  # Ajustar conforme necessário
plt.show()

# Encontrar a melhor técnica de seleção
melhor_selecao_idx = resultados_selecao['Acurácia'].idxmax()
melhor_metodo = resultados_selecao.loc[melhor_selecao_idx, 'Método']
melhor_acc = resultados_selecao.loc[melhor_selecao_idx, 'Acurácia']

# Obter os atributos da melhor técnica de seleção
if melhor_metodo == 'SelectKBest':
    atributos_selecionados = atributos_kbest
elif melhor_metodo == 'RFE':
    atributos_selecionados = atributos_rfe
elif melhor_metodo == 'RandomForest':
    atributos_selecionados = atributos_rf
else:
    atributos_selecionados = X.columns.tolist()

print(f"\nMelhor método de seleção: {melhor_metodo} com acurácia de {melhor_acc:.4f}")
print(f"Atributos selecionados: {atributos_selecionados}")

# Matriz de confusão para o melhor modelo com seleção de atributos
if melhor_metodo == 'SelectKBest':
    cm = confusion_matrix(Y_validation, pred_kbest)
    pred_final = pred_kbest
elif melhor_metodo == 'RFE':
    cm = confusion_matrix(Y_validation, pred_rfe)
    pred_final = pred_rfe
elif melhor_metodo == 'RandomForest':
    cm = confusion_matrix(Y_validation, pred_rf)
    pred_final = pred_rf
else:
    best_model = modelos_dict[melhor_modelo_inicial]
    pred_final = best_model.predict(X_validation)
    cm = confusion_matrix(Y_validation, pred_final)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Insatisfeito', 'Satisfeito'],
           yticklabels=['Insatisfeito', 'Satisfeito'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão - Melhor Modelo')
plt.show()

In [ ]:
## Discussão dos resultados
Nesta seção, interpretamos os resultados obtidos pelos diferentes modelos e técnicas de seleção de atributos.

1. **Comparação de técnicas de seleção de atributos**:
   - O SelectKBest utilizou o teste qui-quadrado para selecionar atributos com maior dependência estatística com a variável alvo.
   - O RFE eliminou recursivamente atributos, mantendo apenas os mais importantes para a classificação.
   - A técnica baseada em Random Forest utilizou a importância dos atributos calculada internamente pelo algoritmo.

2. **Impacto na performance dos modelos**:
   - A seleção de atributos permitiu identificar as características mais importantes para a satisfação dos passageiros.
   - Alguns métodos de seleção conseguiram manter ou até melhorar a performance do modelo com menos atributos.
   - A redução de dimensionalidade simplificou os modelos e potencialmente melhorou a generalização.

3. **Atributos mais importantes**:
   - [Discussão sobre os atributos mais importantes identificados]
   - Esses atributos mostram que a satisfação do passageiro está fortemente relacionada a [fatores específicos como conforto, serviço de bordo, etc.].
   - Esta informação é valiosa para companhias aéreas que desejam melhorar a experiência do cliente.

In [ ]:
## Conclusão

Este trabalho mostrou a aplicação de técnicas de seleção de atributos em um problema de classificação para prever a satisfação de passageiros de companhias aéreas. As principais conclusões são:

1. A seleção de atributos é uma etapa importante no processo de mineração de dados que pode:
   - Reduzir a dimensionalidade dos dados e tornar os modelos mais eficientes
   - Melhorar a interpretabilidade, destacando os fatores mais importantes
   - Potencialmente aumentar a acurácia dos modelos ao eliminar ruídos

2. Diferentes técnicas de seleção de atributos podem identificar conjuntos distintos de características importantes, com algumas sobreposições significativas.

3. O método [melhor método] mostrou-se mais eficaz para este problema, conseguindo [resultado específico].

4. Para empresas aéreas, este estudo mostra que os principais fatores que influenciam a satisfação do cliente são [fatores específicos], o que permite direcionar esforços de melhoria nas áreas mais críticas.

Trabalhos futuros poderiam explorar outras técnicas de seleção de atributos, algoritmos de classificação mais avançados, e validação cruzada para garantir a robustez dos resultados em diferentes segmentos de passageiros.